# Extraktion aller Newsartikel von der Website

Erweitern Sie bitte das in der Demo vorgestellte Beispiel zum Extrahieren der News von der Hochschulseite, um das iterieren aller verfügbaren Newsseiten. Selektieren Sie hierfür mittels Beautiful Soup den "weitere Einträge"-Link und Laden die Folgeseite auf dieser Basis. Fahren Sie solange fohrt, wie dieser Button zur Verfügung steht (Hinweis: max. 10 Seiten).

In [ ]:
import requests
import time
from bs4 import BeautifulSoup

# Methode zum extrahieren der News Informationen (aus Demo entnommen) 
def extract_article_information(article):
    article_info = {}
    article_info["heading"] = article.span.string 
    article_info["teaser"] = article.select_one(".news-article__teaser").text.strip()
    article_info["date"] = article.select_one(".news-article__date").text
    article_info["link"] = article.find("a").attrs["href"]
    return article_info
        
# Trennung der url, da der Link des <a>-Tags von der base_url ausgeht  
base_url = "https://www.hszg.de"
news_url = base_url + "/hochschule/aktuelles/neuigkeiten/page/1"

#Lädt die URL, extrahiert die News und liefert das BeautifulSoup-Objekt zum weiteren selektieren zurück.
def extract_articles_from_page(url):

    # Seite laden und parsen
    request = requests.get(url)
    html = request.content
    soup = BeautifulSoup(html, "html.parser")

    # Artikel Container selektieren
    news_articles = soup.find_all(class_="news-article")
    
    # jeden Artikel im Container extrahieren und der globalen Liste hinzufügen
    for article in news_articles:
        articles.append(extract_article_information(article))
    return soup
        
# Globale Liste instanziiren zum speichern aller Atrikel
articles = []

# erste Seite laden und parsen 
soup = extract_articles_from_page(news_url)

# Direktes extrahieren der ["href"] infos führt zu einem Fehler wenn <a> Tag None ist.
navigation_tag = soup.find("div", class_="news__button-link button--right").a

# Deshalb prüfen wir hier, auf vorhandensein des Links und iterieren, solange dieser verfügbar ist.
while navigation_tag is not None:

    # neuen Abfragelink aus base_url und dem href des <a>-Tags erstellen
    link = base_url + navigation_tag["href"]
    print("Extract content from: " + link)
    # neue Seite vom Server laden
    
    soup = extract_articles_from_page(link)
    
    # wichtig da als Abbruchbedingung genutzt
    navigation_tag = soup.find("div", class_="news__button-link button--right").a
    
    # 3 Sekunden warten
    time.sleep(3)

print(articles)
assert len(articles) == 100, 'Sie haben ' + str(len(articles)) + ' News geladen, aber 100 werden erwartet!'

# Lösungshinweise:

Bei Webdatenextraktion führen wie im wahren Leben, sprichwörtlich viele Wege nach Rom.
Folgende Möglichkeiten zum selektieren des Links wären bspw. denkbar gewesen.

```python
# Varianten zum extrahieren des Links
# 1. div selektieren
next_link_div = soup.find("div", class_="news__button-link button--right")
next_link_div = soup.find_all("div", class_="news__button-link button--right")[0]
next_link_div = soup.select_one("div.news__button-link.button--right")
next_link_div = soup.select("div.news__button-link.button--right")[0]

# 2. <a> separieren bzw Link auslesen
next_link_a = next_link_div.a
next_link_a = next_link_div.find("a")
next_link_a = next_link_div.find_all("a")[0]
next_link_a = next_link_div.select_one("a")
next_link_a = next_link_div.select("a")[0]

# 3. Link extrahieren
next_link = next_link_a["href"]

# alternativ in einem Aufruf durch Kombination z.B.
next_link = soup.find("div", class_="news__button-link button--right").a["href"]
next_link = soup.select_one("div.news__button-link.button--right > a")["href"]
```